In [1]:
%load_ext autoreload
%autoreload 2

# Bellman Equation Q Table Application to Number Guessing Game

This script implements the Bellman equation to train a Q table and test it.

The game I (try to) apply it to is: the numbers 0-4 are shuffled and hidden (i.e. no repeats). As the player, guess numbers (in the range 0-4) until you get first number in the hidden sequence. Then move onto the second number and guess numbers until you get a match. Continue until you've guessed each number in the hidden sequence.

This is a dumb game. And obviously the only strategies are to (1) given a current index, don't repeat guesses and (2) don't guess numbers that you got correct earlier in the sequence.

As you'll see, my first attempt fails completely. The issue is that the state and observation space are useless -- they are both just your current index in the answer sequence. My initial thought was that one only needs the memory of the previous guesses to "beat" this game. Fair enough, but I haven't set this situation up to use that info. Either I need a different algorithm and/or more q-table dimensions OR I need to provide the "AI" with a "memory".

The lesson learned in this first failure is that what you provide in observation/state must be a conscious choice. A thoughtless choice of observation will just probably not be adequate.

Recognizing that in real-world applications, you probably won't be throttling the observation space at all. Anything that could be useful (and which doesn't increase your training time too badly) you want to include.

### Explore NumberGuess Environment

In [6]:
from number_guess import NumberGuess

In [7]:
env = NumberGuess()

print("Random valid answer number:", env.observation_space.sample())
print("Random valid guess number:", env.action_space.sample())


print("\nA few random games:")
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    n_guesses = 0
    while not done:
        n_guesses += 1
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode:{episode} Score:{score} NGuesses:{n_guesses}')

Random valid answer number: 2
Random valid guess number: 4

A few random games:
Episode:1 Score:-17 NGuesses:27
Episode:2 Score:-15 NGuesses:25
Episode:3 Score:-11 NGuesses:21
Episode:4 Score:-18 NGuesses:28
Episode:5 Score:-15 NGuesses:25
Episode:6 Score:-31 NGuesses:41
Episode:7 Score:-10 NGuesses:20
Episode:8 Score:-2 NGuesses:12
Episode:9 Score:-38 NGuesses:48
Episode:10 Score:-5 NGuesses:15


In [94]:
import numpy as np

### Update and training/testing functions

In [95]:
def update_q_table(params, q_table, state_i, state_f, action, step_reward):
    try:
        # note about 2d np array access:
        # q_table[state_i] accesses the state_i-th row, which is an array with
        # length equal to number of actions.
        # q_table[state_i, action] access the action-th element of the state_ith
        # array.
        old_q_value = q_table[state_i, action]
    except IndexError:
        print("ERROR with q_table")
        print(q_table)
        print(state_i, action)
        return None
    
    # max q value given the state after this temp change
    next_max = np.max(q_table[state_f])
    q_target = step_reward + params['gamma'] * next_max
    q_delta = q_target - old_q_value
    q_table[state_i, action] = old_q_value + params['alpha'] * q_delta
    
    return q_table

In [96]:
# alpha - learning rate
# gamma - discount rate
# epsilon - exploration threshold (not currently used)
default_params = {'alpha' : 0.9, 'gamma' : 1., 'epsilon' : 0.}

In [149]:
# loop number_guess games
def train_test(env, in_q_table, n_episodes = 5, do_train = True, params = default_params):
    q_table = in_q_table.copy()
    total_reward = 0
    for i_game in range(n_episodes):
        done = False
        env.reset()
        state_i = env.state
        game_reward = 0
        #print(i_game)
        while not done:
            # choose action
            action = env.action_space.sample()
            if not do_train and game_reward > -20:
                action = np.argmax(q_table[state_i])

            # take a step
            state_f, reward, done, info = env.step(action)
            
            if done:
                continue
            
            #print("  ", env.shower_time, state, reward, done)
            try:
                assert state_f in env.observation_space
            except AssertionError:
                print("Invalid state obtained", state_f, done, i_game, action)
                break
                                        
            # update q table
            if do_train:
                q_table = update_q_table(params, q_table, state_i, state_f, action, reward)

            # increment reward
            game_reward += reward

            state_i = state_f
            
        #print("  Shower reward:", shower_reward)
        total_reward += game_reward

    #np.savetxt("qtable.csv", q_table, delimiter=",")
    avg_reward = total_reward / n_episodes
    return q_table, avg_reward

### Train and Test

In [150]:
# initial q table full of zeroes
init_q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [151]:
# Train
env = NumberGuess(False)
q_table, avg_reward = train_test(env, init_q_table, n_episodes = 100, do_train = True)
print(f"average reward: {avg_reward}")

average reward: -16.32


In [152]:
print(q_table)

[[-20.20924731 -20.27151059 -20.27360285 -20.27360884 -20.15682504]
 [-22.31298974 -22.30130706 -22.30857923 -21.76121376 -22.31309934]
 [-23.8018177  -23.81098675 -23.81098623 -23.80217989 -23.62497236]
 [-26.6153246  -25.6154146  -25.81064852 -26.52540479 -26.60641285]
 [-27.80334063 -26.82235061 -27.82224161 -26.82235062 -27.80235061]]


In [161]:
# Test
avg_reward = train_test(env, q_table, n_episodes = 100, do_train = False)[1]
print(f"average reward: {avg_reward}")
avg_reward = train_test(env, q_table, n_episodes = 100, do_train = False)[1]
print(f"average reward: {avg_reward}")
avg_reward = train_test(env, q_table, n_episodes = 100, do_train = False)[1]
print(f"average reward: {avg_reward}")

average reward: -33.8
average reward: -37.47
average reward: -33.46


Fail! Of course, in hindsight the naive Bellman equation + q table can't get this right. Because the numbers are random, it only recognizes that no number is a good guess. What it /needs/ is memory of its previous guesses, and knowledge of the previous correct answers.

## Small improvement
In this case, a simple and dumb solution to our problem can be to expand the observation space/state. Brainstorming here...

The state could encode:

* knowledge of answer sequence, e.g. [2,4,-1,-1,-1], dim: (5+1)! at most
* guesses in this iteration: [0,0,1,1,0] or some bit rep? 2*5

So this state space dimensionality is: 5! * 10 = 1200

In [12]:
from number_guess import NumberGuess2

In [37]:
env = NumberGuess2()

print("Random valid answer number:", env.observation_space.sample())
print("Random valid guess number:", env.action_space.sample())


print("\nA few random games:")
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    n_guesses = 0
    while not done:
        n_guesses += 1
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode:{episode} Score:{score} NGuesses:{n_guesses}')

Random valid answer number: 64755
Random valid guess number: 0

A few random games:
Episode:1 Score:-4 NGuesses:14
Episode:2 Score:-12 NGuesses:22
Episode:3 Score:-27 NGuesses:37
Episode:4 Score:-6 NGuesses:16
Episode:5 Score:-13 NGuesses:23
Episode:6 Score:-44 NGuesses:54
Episode:7 Score:-30 NGuesses:40
Episode:8 Score:-11 NGuesses:21
Episode:9 Score:-16 NGuesses:26
Episode:10 Score:-12 NGuesses:22
